# When do visibility of files and datasets change?

Files and datasets in LaminDB have the following 3 levels if visibility:
- 1: "default", visible in the UI by default, returned from the query results
- 0: "hidden", excluded from the query results, notebook files (generated by `lamin save`) are default to be hidden
- -1: "trash", excluded from the query results, set with `.delete()`

## Setup

In [ ]:
!lamin init --storage test-visibility

In [ ]:
import lamindb as ln
import pandas as pd

In [ ]:
file = ln.File(pd.DataFrame({"a": [1, 2], "b": [3, 4]}), description="mydf")
file.save()

New file has default visibility 1:

In [ ]:
assert file.visibility == 1

When you delete a file, its visibility is set to -1 ("trash"):

In [ ]:
file.delete()

In [ ]:
assert file.visibility == -1

Files in trash won't be returned from default queries:

In [ ]:
ln.File.filter(description="mydf").all()

Unless you specify `visibility=None` to see all hidden and trashed files:

In [ ]:
ln.File.filter(description="mydf", visibility=None).all()

You can restore a file from trash:

In [ ]:
file.restore()

In [ ]:
assert file.visibility == 1

In [ ]:
ln.File.filter(description="mydf").all()

Delete test file and instance:

In [ ]:
file.delete(permanent=True)

In [ ]:
!lamin delete --force test-visibility